## <font color=yellow>1. 양자화, 모델 로드</font>

In [1]:
!pip install bitsandbytes
!pip install accelerate
!pip install transformers
!pip install peft
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [1]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
)

# !pip install bitsandbytes
config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model_id = "beomi/Llama-3-Open-Ko-8B-Instruct-preview"

tokenizer = AutoTokenizer.from_pretrained(model_id, quantization_config=config)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="cuda:0",
    trust_remote_code=True,
    quantization_config=config
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [3]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params}\nall params: {all_param}\ntrainable%: {100 * trainable_params / all_param}"
    )

In [4]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 3407872
all params: 4544008192
trainable%: 0.07499704789264605


## <font color=yellow>2. 데이터 로드</font>

In [6]:
import huggingface_hub
from huggingface_hub import notebook_login

notebook_login()

In [10]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.7.0
    Uninstalling fsspec-2025.7.0:
      Successfully uninstalled fsspec-2025.7.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.7.0 requires fsspec==2025.7.0, but you have fsspec 2025.3.0 which is incompatible.


In [7]:
from datasets import load_dataset

dataset = load_dataset("nlpai-lab/databricks-dolly-15k-ko")

categories = ['closed_qa', 'information_extraction', 'summarization']
filtered_dataset = dataset['train'].filter(lambda example: example['category'] in categories)

print(filtered_dataset)

README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

databricks-dolly-15k-ko.jsonl:   0%|          | 0.00/15.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15011 [00:00<?, ? examples/s]

Filter:   0%|          | 0/15011 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'context', 'response', 'category', 'id'],
    num_rows: 4529
})


In [8]:
print(set(filtered_dataset['category']))

{'closed_qa', 'summarization', 'information_extraction'}


In [ ]:
SYSTEM_PROMPT = "You are an assistant for answering questions. You are given the extracted parts of a long document and a question. Provide a conversational answer. Don't make up an answer."

tokenizer.pad_token = tokenizer.eos_token

def get_rag_train_prompt(row):
    
    question = "Context에 따르면, " + row['instruction']
    context = row['context']
    answer = row['response']

    user_prompt = f'###Context:{context}\n###Question:{question}'

    messages = [
        {"role": "system", "content" : SYSTEM_PROMPT},
        {"role": "user", "content" : user_prompt},
        {"role": "assistant", "content" : answer}
    ]

    encoded = tokenizer.apply_chat_template(
        messages,
        padding=True,
        truncation=True
    )

    return {"input_ids": encoded}

In [10]:
new_dataset = filtered_dataset.map(get_rag_train_prompt)

Map:   0%|          | 0/4529 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [11]:
print(tokenizer.decode(new_dataset[1]['input_ids']))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an assistant for answering questions. You are given the extracted parts of a long document and a question. Provide a conversational answer. Don't make up an answer.<|eot_id|><|start_header_id|>user<|end_header_id|>

###Context:고모리다는 1981년 7월 10일 구마모토현에서 태어났습니다. 고등학교를 졸업한 후 2000년 J1 리그 클럽 아비스파 후쿠오카에 입단했다. 2001년에 미드필더로 데뷔했지만 많은 경기에 출전하지 못했고, 2001 시즌이 끝나고 클럽은 J2 리그로 강등되었다. 2002년에는 J2 클럽 오이타 트리니타로 이적했다. 그는 수비형 미드필더로 정규 선수가되었고 클럽은 2002 년 우승을 차지했고 2003 년에 승격되었습니다. 그는 2005 년까지 많은 경기를 뛰었습니다. 2005 년 9 월 J2 클럽 몬테 디오 야마가타로 이적했습니다. 2006 년에는 J2 클럽 비셀 고베로 이적했습니다. 수비형 미드필더로 주전으로 활약했지만 여름에는 점차 출전 시간이 줄어들었다. 2007 년에는 자신의 지역에 기반을 둔 일본 축구 리그 클럽 로소 구마모토 (이후 로아소 구마모토)로 이적했습니다. 그는 정규 선수로 뛰었고 클럽은 2008 년에 J2로 승격했습니다. 비록 많은 경기에 출전하지는 못했지만 여전히 많은 경기에 출전했습니다. 2010년에는 인도네시아로 건너가 페르셀라 라몽안에 입단했습니다. 2010년 7월 일본으로 돌아와 J2 클럽 지라반즈 기타큐슈에 입단했습니다. 그는 2012년 은퇴할 때까지 수비형 미드필더와 센터백으로 자주 뛰었습니다.[1][2]
###Question:Context에 따르면, 고모리다 토모아키는 언제 태어났나요?<|eot_id|><|s

## <font color=yellow>3. Train</font>

In [12]:
import transformers

trainer = transformers.Trainer(
    model=model,
    train_dataset=new_dataset,
    args=transformers.TrainingArguments(
        auto_find_batch_size=True,
        gradient_checkpointing=True,
        warmup_steps=10,
        max_steps=2400,
        save_steps=200,
        save_total_limit=3,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rosejeongj1 (rosejeongj1-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


Step,Training Loss
10,2.051500


Step,Training Loss
10,1.702400
20,1.645700
30,1.644400
40,1.594500


Step,Training Loss
10,1.579600


KeyboardInterrupt: 